<iframe src="https://www.tella.tv/video/introduction-to-pandas-ewrj" 
        width="800" 
        height="450" 
        frameborder="0" 
        allowfullscreen></iframe>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import numpy as np
from hashlib import md5
import json


import warnings

warnings.filterwarnings("ignore")

import sys
sys.path.append("..")

from features.feature_engineering import feature_engineering 
from data import get_cleaned_data

# from audit.data import get_cleaned_data

# Set the style

sns.set_style("whitegrid")
pd.set_option("display.max_columns", None)

2025-02-20-18:23:06-rradata.settings-INFO - Loading extra configurations from : /home/shy2351783/.config/rradata/config.yaml


### load_data

In [97]:
df = pd.read_csv('../ressources/Data/audit_data_2016_21.csv').drop(columns=['Unnamed: 0'], errors='ignore')
df_non_audit = pd.read_csv('../ressources/Data/non_audit_data_training.csv').drop(columns=['Unnamed: 0'], errors='ignore')
# df_audit_2016_2021 = pd.read_csv('../ressources/Data/audit_data_2016_21.csv').drop(columns=['Unnamed: 0'], errors='ignore')

In [3]:
df_non_audit.drop(columns=['assess_no_y','assess_no_x'],inplace=True)

## VAT and CIT version

In [35]:
df_vat_cit_audit = df[df['tax_type_no'].isin([1,70,2])]

In [36]:
df_vat_cit_non_audit = df_non_audit[df_non_audit['tax_type_no'].isin([1,70,2])]

In [37]:
df_vat_cit_audit.shape

(7627, 159)

In [38]:
7627/9240

0.8254329004329004

In [39]:
df_vat_cit_non_audit.shape

(18852, 159)

In [40]:
18852/40000

0.4713

#### data understanding 

In [41]:
df_non_audit.shape

(40000, 159)

In [42]:
df.shape

(9240, 159)

In [43]:
df.head()

,doc_no,tp_year,tp_month,tax_type_no,form_no,tax_gap,audit_type,tp_start_date,tp_end_date,tp_due_date,submit_date,exempted_sales,zero_rated_sales,exports,total_not_taxable,taxable_sales_subject_to_vat,vat_on_taxable_sales,vat_reverse_charge,vat_payable,vat_paid_on_imports,vat_paid_on_local_purchases,total_vat_paid_on_inputs,vat_reverse_charge_deductible,vat_payable_credit_refundable,credit_carried_from_previous_month,total_amount_on_invoices_to_public_institutions,vat_withholding_taken_by_public_institutions,vat_due_credit_payable,vat_refund_paid_carried_forward_offset,number_employees,total_basic_pay,total_allowance_paid,total_benefit_in_kind,other_taxable_payments,nsf_contribution_by_employees,nsf_contribution_by_employer,non_taxable_deductions,calc_taxable_pay,tax_due,number_casual_employees,total_remuneration_paid,tax_due_apply_rate_apply_law,number_of_employees,total_remuneration_paid_2,paye_withheld,total_calculation_base_for_pension,total_calculation_base_for_medial,total_calculation_base_for_maternity,it_opening_stock,it_purchases,it_closing_stock,it_gross_profit,it_depreciation,it_total_expenses_depreciation,it_net_operating_income,it_investment_income,it_non_operating_income,it_rental_income,it_total_income,it_investment_expenses,it_non_operational_expenses,it_training_expenses,it_investment_allowance,it_bad_debts,it_rental_expenses,it_tax_paid_on_minerals,it_total_deductions,it_net_income,it_reintegration_nondeductible_expenses,it_depreciation_adjustments,it_loss_carried_forward,it_non_taxable_dividend,it_taxable_income_loss,it_corporate_income_tax_to_pay,it_tax_discount_from_employment,it_tax_discount_from_exports,it_tax_discount_free_trade_zone,it_tax_discount_micro_finance,it_tax_discount_newly_listed_companies,it_tax_discount_venture_capital,it_collective_investment_scheme,it_tax_discount_agriculture,it_total_discounts,it_foreign_tax_credit,it_corporate_income_tax_payable,it_quarterly_prepayments,it_withholding_on_imports,it_withholding_public_supplies,it_withholding_other_payments,it_total_credits,it_overpayment_previous_periods,it_net_tax_due_credit,it_exempted_income,it_taxable_income,it_corporate_income,it_withholding_on_public_tenders,it_net_tax_due,it_credit_approved,it_credit_used,it_credit_added_back,it_tax_income_year,it_annual_flat_amount_tax_due,it_tax_on_sales_immovable_property,it_purchases_supported_by_ebm_dmc_wop,it_employment_income,it_transport_income,it_non_operating_and_extraordinary_income,it_reintegration_of_non_deductible_expenses,it_personal_income_tax,it_personal_income_tax_payable,it_withholding_on_public_supplies,it_withholding_on_other_payments,it_withholding_on_payments_paye,it_overpayments_from_previous_iqp,it_net_tax_due_or_credit,tax_centre_no,district_no,tp_size,tax_type_no_count,reg_date,close_date,seg_no,tax_type_desc,tax_to_pay,pen_to_pay,int_to_pay,tax_paid,pen_paid,int_paid,balance,ebm_purchases,ebm_taxable_purchases,ebm_exempted_purchases,ebm_zero_rated_purchases,ebm_purchases_vat,invoice_no_purchases,insurance,cif,total_taxes,gross_weight,net_weight,fob_amount,ebm_sales,ebm_taxable_sales,ebm_exempted_sales,ebm_zero_rated_sales,ebm_vat,invoice_no_sales,tax_to_pay_declaration,turnovers,refund_claimed,it_cost_of_goods,it_operating_expenses_ebm_dmc,it_expenses_related_salaries,it_expenses_supported_by_withholdings,it_expenses_not_requiring_ebm_or_dmc,it_operating_expenses,isic_desc,hashed_tin
0,47644537,2018,12.0,1,9,83740.0,3,2018-12-01,2018-12-31,2019-01-15,2019-01-14 12:56:17,200.0,0.0,0.0,200.0,82147551.0,14786559.0,0.0,14786559.0,0.0,14758892.0,14758892.0,0.0,27667.0,0.0,0.0,0.0,27667.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,120,S,3,2013-05-21 17:23:42,2018-02-28 00:

In [44]:
df_non_audit.head()

,tax_type_no,form_no,doc_no,tp_year,tp_month,tp_start_date,tp_end_date,tp_due_date,submit_date,exempted_sales,zero_rated_sales,exports,total_not_taxable,taxable_sales_subject_to_vat,vat_on_taxable_sales,vat_reverse_charge,vat_payable,vat_paid_on_imports,vat_paid_on_local_purchases,total_vat_paid_on_inputs,vat_reverse_charge_deductible,vat_payable_credit_refundable,credit_carried_from_previous_month,total_amount_on_invoices_to_public_institutions,vat_withholding_taken_by_public_institutions,vat_due_credit_payable,vat_refund_paid_carried_forward_offset,number_employees,total_basic_pay,total_allowance_paid,total_benefit_in_kind,other_taxable_payments,nsf_contribution_by_employees,nsf_contribution_by_employer,non_taxable_deductions,calc_taxable_pay,tax_due,number_casual_employees,total_remuneration_paid,tax_due_apply_rate_apply_law,number_of_employees,total_remuneration_paid_2,paye_withheld,total_calculation_base_for_pension,total_calculation_base_for_medial,total_calculation_base_for_maternity,it_opening_stock,it_purchases,it_closing_stock,it_gross_profit,it_depreciation,it_total_expenses_depreciation,it_net_operating_income,it_investment_income,it_non_operating_income,it_rental_income,it_total_income,it_investment_expenses,it_non_operational_expenses,it_training_expenses,it_investment_allowance,it_bad_debts,it_rental_expenses,it_tax_paid_on_minerals,it_total_deductions,it_net_income,it_reintegration_nondeductible_expenses,it_depreciation_adjustments,it_loss_carried_forward,it_non_taxable_dividend,it_taxable_income_loss,it_corporate_income_tax_to_pay,it_tax_discount_from_employment,it_tax_discount_from_exports,it_tax_discount_free_trade_zone,it_tax_discount_micro_finance,it_tax_discount_newly_listed_companies,it_tax_discount_venture_capital,it_collective_investment_scheme,it_tax_discount_agriculture,it_total_discounts,it_foreign_tax_credit,it_corporate_income_tax_payable,it_quarterly_prepayments,it_withholding_on_imports,it_withholding_public_supplies,it_withholding_other_payments,it_total_credits,it_overpayment_previous_periods,it_net_tax_due_credit,it_exempted_income,it_taxable_income,it_corporate_income,it_withholding_on_public_tenders,it_net_tax_due,it_credit_approved,it_credit_used,it_credit_added_back,assess_no_x,it_tax_income_year,it_annual_flat_amount_tax_due,it_tax_on_sales_immovable_property,it_purchases_supported_by_ebm_dmc_wop,it_employment_income,it_transport_income,it_non_operating_and_extraordinary_income,it_reintegration_of_non_deductible_expenses,it_personal_income_tax,it_personal_income_tax_payable,it_withholding_on_public_supplies,it_withholding_on_other_payments,it_withholding_on_payments_paye,it_overpayments_from_previous_iqp,it_net_tax_due_or_credit,assess_no_y,tax_type_desc,tax_to_pay,pen_to_pay,int_to_pay,tax_paid,pen_paid,int_paid,balance,tax_centre_no,district_no,tp_size,tax_type_no_count,reg_date,close_date,seg_no,insurance,cif,total_taxes,gross_weight,net_weight,fob_amount,ebm_purchases,ebm_taxable_purchases,ebm_exempted_purchases,ebm_zero_rated_purchases,ebm_purchases_vat,invoice_no_purchases,ebm_sales,ebm_taxable_sales,ebm_exempted_sales,ebm_zero_rated_sales,ebm_vat,invoice_no_sales,tax_to_pay_declaration,turnovers,refund_claimed,it_cost_of_goods,it_operating_expenses_ebm_dmc,it_expenses_related_salaries,it_expenses_supported_by_withholdings,it_expenses_not_requiring_ebm_or_dmc,it_operating_expenses,isic_desc,hashed_tin
0,1,9,52695011,2020,12.0,2020-12-01,2020-12-31,2021-01-15,2021-01-08 10:10:40,1563539.0,0.0,0.0,1563539.0,30122243.0,5422004.0,0.0,5422004.0,0.0,5369755.0,5369755.0,0.0,52249.0,0.0,0.0,0.0,52249.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16109514.0,VAT-VALUE ADDED TAX,52249.0,0.0

In [45]:
df_columns = pd.DataFrame(df.columns.tolist(), columns=['variables'])


In [46]:
df_columns.to_excel('../ressources/Data/audit_dataset_variables.xlsx')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Columns: 159 entries, doc_no to hashed_tin
dtypes: float64(140), int64(9), object(10)
memory usage: 11.2+ MB


In [50]:
df_non_audit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 159 entries, tax_type_no to hashed_tin
dtypes: float64(145), int64(4), object(10)
memory usage: 48.5+ MB


In [51]:
(df.isna().mean() * 100).sort_values(ascending=False).head(50)

it_tax_on_sales_immovable_property             99.935065
it_annual_flat_amount_tax_due                  99.859307
it_tax_income_year                             99.859307
it_tax_discount_free_trade_zone                99.675325
it_tax_discount_newly_listed_companies         99.675325
it_collective_investment_scheme                99.675325
it_tax_discount_agriculture                    99.675325
it_tax_discount_from_employment                99.675325
it_tax_discount_venture_capital                99.675325
it_tax_discount_from_exports                   99.675325
it_purchases_supported_by_ebm_dmc_wop          98.225108
it_net_tax_due_or_credit                       98.225108
it_withholding_on_public_supplies              98.225108
it_overpayments_from_previous_iqp              98.225108
it_withholding_on_payments_paye                98.225108
it_withholding_on_other_payments               98.225108
it_personal_income_tax                         98.225108
it_reintegration_of_non_deducti

In [52]:
(df_vat_cit_audit.isna().mean() * 100).sort_values(ascending=False).head(50)

it_annual_flat_amount_tax_due                  100.000000
number_employees                               100.000000
total_calculation_base_for_medial              100.000000
total_calculation_base_for_pension             100.000000
paye_withheld                                  100.000000
total_remuneration_paid_2                      100.000000
number_of_employees                            100.000000
tax_due_apply_rate_apply_law                   100.000000
total_remuneration_paid                        100.000000
total_basic_pay                                100.000000
it_tax_income_year                             100.000000
total_allowance_paid                           100.000000
total_benefit_in_kind                          100.000000
other_taxable_payments                         100.000000
nsf_contribution_by_employees                  100.000000
nsf_contribution_by_employer                   100.000000
non_taxable_deductions                         100.000000
calc_taxable_p

In [53]:
(df_non_audit.isna().mean() * 100).sort_values(ascending=False).head(50)

it_tax_discount_agriculture                    100.0000
it_tax_discount_from_exports                   100.0000
it_collective_investment_scheme                100.0000
it_tax_discount_venture_capital                100.0000
it_tax_discount_newly_listed_companies         100.0000
it_tax_discount_free_trade_zone                100.0000
it_tax_discount_from_employment                100.0000
it_tax_on_sales_immovable_property              99.6950
fob_amount                                      99.0200
it_tax_discount_micro_finance                   98.8275
it_withholding_on_other_payments                97.7000
it_withholding_on_public_supplies               97.7000
it_personal_income_tax_payable                  97.6975
it_non_operating_and_extraordinary_income       97.6850
it_purchases_supported_by_ebm_dmc_wop           97.6850
it_transport_income                             97.6850
it_employment_income                            97.6850
it_reintegration_of_non_deductible_expenses     

In [54]:
column_all_nan = ['it_collective_investment_scheme','it_tax_discount_from_employment','it_tax_discount_free_trade_zone','it_tax_discount_newly_listed_companies','it_tax_discount_agriculture']

In [55]:
df_non_audit[column_all_nan]

,it_collective_investment_scheme,it_tax_discount_from_employment,it_tax_discount_free_trade_zone,it_tax_discount_newly_listed_companies,it_tax_discount_agriculture
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
39995,NaN,NaN,NaN,NaN,NaN
39996,NaN,NaN,NaN,NaN,NaN
39997,NaN,NaN,NaN,NaN,NaN
39998,NaN,NaN,NaN,NaN,NaN


In [56]:
it_collective_investment_scheme                100.0000
it_tax_discount_from_employment                100.0000
it_tax_discount_free_trade_zone                100.0000
it_tax_discount_newly_listed_companies         100.0000
it_tax_discount_venture_capital                100.0000
it_tax_discount_agriculture                    100.0000
it_tax_discount_from_exports                   100.0000

SyntaxError: invalid syntax (3122007484.py, line 1)

The 100% NaN values in non-audit declarations come from Income Tax declarations (real regime), where the corresponding lines are missing in the declaration. However, in audit declarations, the same columns that contain NaN values in non-audit declarations have zeros instead.

In [57]:
df.query('it_tax_discount_agriculture.isna()')[['it_collective_investment_scheme','hashed_tin','tp_year']]

,it_collective_investment_scheme,hashed_tin,tp_year
0,NaN,77169e4fed9df1980db2f67c4b53ade1,2018
1,NaN,dab7526358dad4f90cf16d8afa957a6f,2018
2,NaN,36a7f5dd5f4aff9aa9a66ec0cc1d2b2a,2018
3,NaN,8ce84a2fac2280353106beb9e1f78039,2018
4,NaN,172b459a6c2765ea207e3b2575516d6c,2019
...,...,...,...
9235,NaN,e729ada509562193243699cc2972984c,2020
9236,NaN,25a3d9d6e0f52f6704637beeb61b7362,2020
9237,NaN,76d51fb3977ee2f34eb2d6a03b567678,2020
9238,NaN,369eb3b0f36568fb0228a2d995a63d8f,2021


In [58]:
summary_stats = df.select_dtypes(include=['float']).describe()

In [59]:
summary_stats.to_excel('../ressources/Data/summary_statistics.xlsx')

In [60]:
((3.610950e+05 * 1000)/ 3600000000000) * 100

0.010030416666666667

In [61]:
df_non_audit.select_dtypes(include=['float']).describe()

,tp_month,exempted_sales,zero_rated_sales,exports,total_not_taxable,taxable_sales_subject_to_vat,vat_on_taxable_sales,vat_reverse_charge,vat_payable,vat_paid_on_imports,vat_paid_on_local_purchases,total_vat_paid_on_inputs,vat_reverse_charge_deductible,vat_payable_credit_refundable,credit_carried_from_previous_month,total_amount_on_invoices_to_public_institutions,vat_withholding_taken_by_public_institutions,vat_due_credit_payable,vat_refund_paid_carried_forward_offset,number_employees,total_basic_pay,total_allowance_paid,total_benefit_in_kind,other_taxable_payments,nsf_contribution_by_employees,nsf_contribution_by_employer,non_taxable_deductions,calc_taxable_pay,tax_due,number_casual_employees,total_remuneration_paid,tax_due_apply_rate_apply_law,number_of_employees,total_remuneration_paid_2,paye_withheld,total_calculation_base_for_pension,total_calculation_base_for_medial,total_calculation_base_for_maternity,it_opening_stock,it_purchases,it_closing_stock,it_gross_profit,it_depreciation,it_total_expenses_depreciation,it_net_operating_income,it_investment_income,it_non_operating_income,it_rental_income,it_total_income,it_investment_expenses,it_non_operational_expenses,it_training_expenses,it_investment_allowance,it_bad_debts,it_rental_expenses,it_tax_paid_on_minerals,it_total_deductions,it_net_income,it_reintegration_nondeductible_expenses,it_depreciation_adjustments,it_loss_carried_forward,it_non_taxable_dividend,it_taxable_income_loss,it_corporate_income_tax_to_pay,it_tax_discount_from_employment,it_tax_discount_from_exports,it_tax_discount_free_trade_zone,it_tax_discount_micro_finance,it_tax_discount_newly_listed_companies,it_tax_discount_venture_capital,it_collective_investment_scheme,it_tax_discount_agriculture,it_total_discounts,it_foreign_tax_credit,it_corporate_income_tax_payable,it_quarterly_prepayments,it_withholding_on_imports,it_withholding_public_supplies,it_withholding_other_payments,it_total_credits,it_overpayment_previous_periods,it_net_tax_due_credit,it_exempted_income,it_taxable_income,it_corporate_income,it_withholding_on_public_tenders,it_net_tax_due,it_credit_approved,it_credit_used,it_credit_added_back,assess_no_x,it_tax_income_year,it_annual_flat_amount_tax_due,it_tax_on_sales_immovable_property,it_purchases_supported_by_ebm_dmc_wop,it_employment_income,it_transport_income,it_non_operating_and_extraordinary_income,it_reintegration_of_non_deductible_expenses,it_personal_income_tax,it_personal_income_tax_payable,it_withholding_on_public_supplies,it_withholding_on_other_payments,it_withholding_on_payments_paye,it_overpayments_from_previous_iqp,it_net_tax_due_or_credit,assess_no_y,tax_to_pay,pen_to_pay,int_to_pay,tax_paid,pen_paid,int_paid,balance,tax_centre_no,district_no,tax_type_no_count,seg_no,insurance,cif,total_taxes,gross_weight,net_weight,fob_amount,ebm_purchases,ebm_taxable_purchases,ebm_exempted_purchases,ebm_zero_rated_purchases,ebm_purchases_vat,invoice_no_purchases,ebm_sales,ebm_taxable_sales,ebm_exempted_sales,ebm_zero_rated_sales,ebm_vat,invoice_no_sales,tax_to_pay_declaration,turnovers,refund_claimed,it_cost_of_goods,it_operating_expenses_ebm_dmc,it_expenses_related_salaries,it_expenses_supported_by_withholdings,it_expenses_not_requiring_ebm_or_dmc,it_operating_expenses
count,25575.000000,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,1.262200e+04,12622.0,1.262200e+04,1.262200e+04,12622.0,12953.000000,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,12953.000000,1.295300e+04,1.295300e+04,12953.000000,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,1.295300e+04,3.078000e+03,2.152000e+03,3.078000e+03,3.078000e+03,2.152000e+03,2.152000e+03,3.078000e+03,3.078000e+03,2.152000e+03,2.153000e+03,3.079000e+03,2.153000e+03,2.153000e+03,2.153000e+03,2.153000e+03,2.153000e+03,2.153000e+03,2153.000000,3.079000e+03,3.0

# target variable (tax gap) 

In [2]:
def plot_target_distribution_pie_chart(data_column):
    """
    Plots a pie chart showing the distribution of values in the target variable
    based on the categories: Above 0, Equal to 0, and Below 0.

    Args:
    data_column (pd.Series): A pandas Series containing the data to analyze.
    """
    # Categorize values
    above_zero = data_column > 0
    equal_zero = data_column == 0
    below_zero = data_column < 0

    # Count each category
    categories = {
        "Above 0": above_zero.sum(),
        "Below 0": below_zero.sum(),
    }

    # Calculate percentages
    total = len(data_column)
    percentages = {key: (value / total) * 100 for key, value in categories.items()}

    # Plot the pie chart
    plt.figure(figsize=(4, 3))
    plt.pie(
        percentages.values(),
        labels=percentages.keys(),
        autopct="%1.1f%%",
        startangle=140,
        colors=["green", "blue", "red"],
    )
    plt.title("Distribution of different categories of tax gap")
    plt.show()

In [3]:
plot_target_distribution_pie_chart(df["tax_gap"])

NameError: name 'df' is not defined

In [4]:
plot_target_distribution_pie_chart(df_audit_2016_2021["tax_gap"])

NameError: name 'df_audit_2016_2021' is not defined

In [5]:
plot_target_distribution_pie_chart(df_vat_cit_audit["tax_gap"])

NameError: name 'df_vat_cit_audit' is not defined

The pie chart illustrates the distribution of the target variable, tax_gap, which represents the difference between the principal tax due (determined after auditing) and the self-assessed tax due (declared by taxpayers). The findings reveal that 90.9% of audited declarations result in an increase in the tax due compared to the amounts declared by taxpayers. Additionally, 7.2% of the declarations show no change in the tax due, while 1.9% indicate a reduction in the tax due following the audit.

In [ ]:
def audit_desc(audit_type):
    """
    this function will map audit type to their audit type description

    Parameters
    ---------

    """
    audit_desc = {
        1: "Advisory",
        2: "Desk Audit",
        3: "Comprehensive",
        4: "Issue oriented",
        5: "Refund",
        6: "Record keeping check",
    }
    return audit_desc.get(audit_type, None)

In [ ]:
### create a column of audit type description
df["audit_type_desc"] = df["audit_type"].apply(lambda x: audit_desc(x))

In [ ]:
df_vat_cit_audit["audit_type_desc"] = df["audit_type"].apply(lambda x: audit_desc(x))

In [ ]:
df.audit_type_desc.unique()

In [ ]:
def plot_tax_gap_by_audit_type(df, tax_gap_col, audit_type_col):
    """
    Plots a clustered bar chart showing the distribution of tax gap categories
    (Increase, No Change, Decrease) across different audit types.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    tax_gap_col (str): Column name representing the tax gap values.
    audit_type_col (str): Column name representing the audit types.
    """
    # Categorize tax_gap into three categories
    df["Tax Gap Category"] = df[tax_gap_col].apply(
        lambda x: "Increase" if x > 0 else ("No Change" if x == 0 else "Decrease")
    )

    # Group data by Tax Gap Category and Audit Type, then count occurrences
    grouped_data = (
        df.groupby(["Tax Gap Category", audit_type_col]).size().unstack(fill_value=0)
    )

    # Plotting the chart
    grouped_data.plot(kind="bar", figsize=(10, 6), stacked=False)
    plt.title(f"Distribution of Tax Gap Categories by {audit_type_col}")
    plt.xlabel("Tax Gap Category")
    plt.ylabel("Count")
    plt.legend(title=f"{audit_type_col}", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.xticks(rotation=0)
    plt.tight_layout()  # Adjust layout to prevent cutting off labels
    plt.show()

In [ ]:
# Plotting the chart
plot_tax_gap_by_audit_type(df, tax_gap_col="tax_gap", audit_type_col="audit_type_desc")

The graph above illustrates the distribution of three categories based on their audit type. As shown, the "increase in tax due" category, which represents about 91% of all declarations, is predominantly associated with desk audits. In contrast, the other categories are mainly linked to comprehensive audits.

It's important to note that comprehensive audits can cover extended tax periods, such as three years, and involve multiple tax types. As a result, while a decrease in one tax type may be identified, an increase in another could be found, leading to taxpayers owing additional amounts, including fines, after the audit.

In [ ]:
plot_tax_gap_by_audit_type(df_vat_cit_audit, tax_gap_col="tax_gap", audit_type_col="audit_type_desc")

In [ ]:
def visualize_distribution_and_outliers(df, target_col):
    """
    Visualizes the distribution of the target variable using a histogram
    and a box plot side by side.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    target_col (str): Column name of the target variable to be visualized.
    """
    # Remove NaN values and filter out any extreme outliers for better visualization
    target_data = df[target_col].dropna()

    # Create a figure with two subplots (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Histogram (left plot)
    axes[0].hist(target_data, bins=20, color="skyblue", edgecolor="black")
    axes[0].set_title(f"{target_col} Distribution (Histogram)")
    axes[0].set_xlabel(target_col)
    axes[0].set_ylabel("Frequency")

    # Box plot (right plot)
    axes[1].boxplot(
        target_data,
        vert=False,
        patch_artist=True,
        boxprops=dict(facecolor="lightgreen", color="black"),
        medianprops=dict(color="red"),
    )
    axes[1].set_title(f"{target_col} Distribution (Box Plot)")
    axes[1].set_xlabel(target_col)

    # Adjust layout for better appearance
    plt.tight_layout()
    plt.show()

### Distribution of target variable (yield) by tax type and taxpayer size 

In [ ]:
def visualize_distribution_and_outliers_hue(df, target_col, hue_col=None):
    """
    Visualizes the distribution of the target variable using a histogram
    and a box plot for each distinct value in hue_col.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    target_col (str): Column name of the target variable to be visualized.
    hue_col (str, optional): Column name of the variable for creating separate plots.
    """
    if hue_col:
        # Get unique values for hue
        hues = df[hue_col].dropna().unique()
        
        # Calculate number of rows based on the number of unique hues
        n_rows = len(hues)
        fig, axes = plt.subplots(n_rows, 2, figsize=(14, 6 * n_rows))

        # If there's only one hue value, axes will be a 1D array, so reshape it
        if n_rows == 1:
            axes = axes.reshape(1, -1)

        for i, hue_value in enumerate(hues):
            # Create a subset DataFrame for the current hue value
            subset_df = df[df[hue_col] == hue_value].dropna(subset=[target_col])
            
            # Histogram
            sns.histplot(data=subset_df, x=target_col, bins=20, ax=axes[i, 0], edgecolor="black")
            axes[i, 0].set_title(f"{target_col} Distribution for {hue_col} = {hue_value}")
            axes[i, 0].set_xlabel(target_col)
            axes[i, 0].set_ylabel("Frequency")

            # Box plot
            sns.boxplot(data=subset_df, x=target_col, ax=axes[i, 1], 
                        boxprops=dict(facecolor="lightgreen", color="black"),
                        medianprops=dict(color="red"))
            axes[i, 1].set_title(f"{target_col} Box Plot for {hue_col} = {hue_value}")
            axes[i, 1].set_xlabel(target_col)

    else:
        # If no hue, use the original plotting logic
        data = df[target_col].dropna()
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))

        # Histogram
        axes[0].hist(data, bins=20, color="skyblue", edgecolor="black")
        axes[0].set_title(f"{target_col} Distribution (Histogram)")
        axes[0].set_xlabel(target_col)
        axes[0].set_ylabel("Frequency")

        # Box plot
        axes[1].boxplot(data, vert=False, patch_artist=True,
                        boxprops=dict(facecolor="lightgreen", color="black"),
                        medianprops=dict(color="red"))
        axes[1].set_title(f"{target_col} Distribution (Box Plot)")
        axes[1].set_xlabel(target_col)

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_distribution_and_outliers_hue(df, "tax_gap","tp_size")

In [ ]:
visualize_distribution_and_outliers_hue(df_vat_cit_audit, "tax_gap","tp_size")

In [ ]:
visualize_distribution_and_outliers_hue(df, "tax_gap","tax_type_no")

In [ ]:
visualize_distribution_and_outliers_hue(df_vat_cit_audit, "tax_gap","tax_type_no")

In [ ]:
visualize_distribution_and_outliers_hue(df, "tax_gap","tp_size")

In [ ]:
visualize_distribution_and_outliers(df, "tax_gap")

The plots above illustrate the distribution of our target variable. The histogram reveals that the tax gap data is right-skewed, indicating the presence of positive outliers (higher values), which is further confirmed by the box plot on the right. This skewness could be attributed to variations in tax types; for example, the tax gap in income tax may be significantly higher compared to VAT on a monthly basis.

### analyzing outliers 

In [ ]:
def filter_outliers_by_hue(df, column, hue=None, iqr_factor=1.5):
    """
    Filter outliers from a DataFrame based on a specified column using IQR method,
    optionally segmented by a hue column.

    Parameters:
    - df: pandas DataFrame containing the data
    - column: string, name of the column to check for outliers
    - hue: string, optional, name of the column to segment the data by
    - iqr_factor: float, factor to multiply IQR for determining bounds

    Returns:
    - pandas DataFrame with only the outliers and their computed bounds
    """
    if hue is None:
        return _filter_outliers(df, column, iqr_factor)
    else:
        outlier_dfs = []
        
        for group, data in df.groupby(hue):
            group_outliers = _filter_outliers(data, column, iqr_factor)
            group_outliers[hue] = group
            outlier_dfs.append(group_outliers)
        
        return pd.concat(outlier_dfs, ignore_index=True)

def _filter_outliers(df, column, iqr_factor):
    """
    Helper function to filter outliers using IQR in one segment of data.
    """
    data = df[column]
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - (iqr_factor * IQR)
    upper_bound = Q3 + (iqr_factor * IQR)
    
    outliers = df[(data < lower_bound) | (data > upper_bound)]
    
    # Add bounds information to the outliers DataFrame
    outliers['lower_bound'] = lower_bound
    outliers['upper_bound'] = upper_bound
    
    return outliers

### outliers 

In [ ]:
df_size_outliers = filter_outliers_by_hue(df,"tax_gap",hue="tp_size")

In [6]:
df_type_outliers = filter_outliers_by_hue(df,"tax_gap",hue="tax_type_no")

NameError: name 'filter_outliers_by_hue' is not defined

In [ ]:
df_size_outliers_version = filter_outliers_by_hue(df_vat_cit_audit,"tax_gap",hue="tp_size")

In [ ]:
df_type_outliers_version = filter_outliers_by_hue(df_vat_cit_audit,"tax_gap",hue="tax_type_no")

### remove outliers 

In [ ]:
df_with_no_outliers = df[~(df['hashed_tin'].isin(df_size_outliers['hashed_tin']))]

In [ ]:
df_with_type_no_outliers = df[~(df['hashed_tin'].isin(df_type_outliers['hashed_tin']))]

In [ ]:
df_with_no_outliers_version =  df_vat_cit_audit[~(df_vat_cit_audit['hashed_tin'].isin(df_size_outliers_version['hashed_tin']))]

In [ ]:
df_with_type_no_outliers_version =  df_vat_cit_audit[~(df_vat_cit_audit['hashed_tin'].isin(df_type_outliers_version['hashed_tin']))]

In [ ]:
visualize_distribution_and_outliers_hue(df_with_no_outliers, "tax_gap","tp_size")

In [ ]:
visualize_distribution_and_outliers_hue(df_with_no_outliers_version, "tax_gap","tp_size")

In [ ]:
visualize_distribution_and_outliers_hue(df_with_type_no_outliers, "tax_gap","tax_type_no")

In [ ]:
visualize_distribution_and_outliers_hue(df_with_type_no_outliers_version, "tax_gap","tax_type_no")

## tax type , District number, year vs tax gap 

In [ ]:
def tax_type_desc(tax_type_no):
    """
    this function will map audit type to their audit type description

    Parameters
    ---------

    """
    tax_desc = {
        1: "VAT monthly",
        70: "VAT quarterly",
        2: "CIT",
        8: "PIT ",
        9: "PAYE",
        77: "PIT Flat",
        78: "CIT flat",
    }
    return tax_desc.get(tax_type_no, None)

In [ ]:
df["tax_type_desc"] = df["tax_type_no"].apply(lambda x: tax_type_desc(x))

In [ ]:
df_non_audit['tax_type_desc'] = df_non_audit['tax_type_no'].apply(lambda x: tax_type_desc(x))

In [ ]:
df_vat_cit_audit['tax_type_desc'] = df_vat_cit_audit['tax_type_no'].apply(lambda x: tax_type_desc(x))

In [ ]:
def visualize_tax_type_and_tax_gap(
    df, tax_type_col, tax_gap_col, orientation="vertical"
):
    """
    Visualizes the Audited Declarations of tax types and the associated tax gap values side by side.
    You can choose between vertical and horizontal bar charts and display percentages on each bar.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    tax_type_col (str): Column name of the tax type.
    tax_gap_col (str): Column name of the tax gap.
    orientation (str): Orientation of the bar charts ('vertical' or 'horizontal').
    """
    # Calculate the Audited Declarations of each tax_type
    tax_type_freq = df[tax_type_col].value_counts()

    # Calculate the total tax_gap for each tax_type
    tax_gap_sum = df.groupby(tax_type_col)[tax_gap_col].sum()

    # Sort the tax_gap sums in descending order
    tax_gap_sum = tax_gap_sum.sort_values(ascending=False)

    # Create a figure with two subplots (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # For percentages
    tax_type_freq_percentage = tax_type_freq / tax_type_freq.sum() * 100
    tax_gap_sum_percentage = tax_gap_sum / tax_gap_sum.sum() * 100

    if orientation == "horizontal":
        # Plot Audited Declarations (left side) - horizontal
        tax_type_freq.plot(kind="barh", color="skyblue", edgecolor="black", ax=axes[0])
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel("Audited Declarations")
        axes[0].set_ylabel(f"{tax_type_col}")

        # Annotate percentages on bars
        for i, v in enumerate(tax_type_freq):
            axes[0].text(v + 1, i, f"{tax_type_freq_percentage[i]:.1f}%", va="center")

        # Plot tax_gap sum (right side) - horizontal
        tax_gap_sum.plot(kind="barh", color="lightgreen", edgecolor="black", ax=axes[1])
        axes[1].set_title(f"{tax_gap_col} by {tax_type_col}")
        axes[1].set_xlabel(f"{tax_gap_col}")
        axes[1].set_ylabel(f"{tax_type_col}")

        # Annotate percentages on bars
        for i, v in enumerate(tax_gap_sum):
            axes[1].text(v + 1, i, f"{tax_gap_sum_percentage[i]:.1f}%", va="center")
    else:
        # Plot Audited Declarations (left side) - vertical
        tax_type_freq.plot(kind="bar", color="skyblue", edgecolor="black", ax=axes[0])
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel(f"{tax_type_col}")
        axes[0].set_ylabel("Audited Declarations")

        try:
            # Annotate percentages on bars
            for i, v in enumerate(tax_type_freq):
                axes[0].text(
                    i, v + 1, f"{tax_type_freq_percentage[i]:.1f}%", ha="center"
                )

        except:

            pass

        # Plot tax_gap sum (right side) - vertical
        tax_gap_sum.plot(kind="bar", color="lightgreen", edgecolor="black", ax=axes[1])
        axes[1].set_title(f"{tax_gap_col} by {tax_type_col}")
        axes[1].set_xlabel(f"{tax_type_col}")
        axes[1].set_ylabel(f"{tax_gap_col}")

        try:
            # Annotate percentages on bars
            for i, v in enumerate(tax_gap_sum):
                axes[1].text(i, v + 1, f"{tax_gap_sum_percentage[i]:.1f}%", ha="center")

        except:

            pass

    # Adjust layout for better appearance
    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_tax_type_and_tax_gap_hue(
    df, tax_type_col, tax_gap_col, orientation="vertical", hue_col=None
):
    """
    Visualizes the Audited Declarations of tax types and the associated tax gap values side by side.
    You can choose between vertical and horizontal bar charts.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    tax_type_col (str): Column name of the tax type.
    tax_gap_col (str): Column name of the tax gap.
    orientation (str): Orientation of the bar charts ('vertical' or 'horizontal').
    hue_col (str, optional): Column name to add hue for further grouping (clustered bars).
    """
    # Prepare data
    if hue_col:
        tax_type_freq = df.groupby([tax_type_col, hue_col]).size().unstack(fill_value=0)
        tax_gap_sum = (
            df.groupby([tax_type_col, hue_col])[tax_gap_col].sum().unstack(fill_value=0)
        )
    else:
        tax_type_freq = df[tax_type_col].value_counts()
        tax_gap_sum = df.groupby(tax_type_col)[tax_gap_col].sum()

    # Sort the tax_gap sums in descending order if not using hue_col
    if not hue_col:
        tax_gap_sum = tax_gap_sum.sort_values(ascending=False)

    # Create a figure with two subplots (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    if orientation == "horizontal":
        # Plot Audited Declarations (left side) - horizontal
        if hue_col:
            tax_type_freq.plot(kind="barh", color=plt.cm.tab20.colors, ax=axes[0])
        else:
            tax_type_freq.plot(
                kind="barh", color="skyblue", edgecolor="black", ax=axes[0]
            )
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel("Audited Declarations")
        axes[0].set_ylabel(f"{tax_type_col}")

        # Plot tax_gap sum (right side) - horizontal
        if hue_col:
            tax_gap_sum.plot(kind="barh", color=plt.cm.tab20.colors, ax=axes[1])
        else:
            tax_gap_sum.plot(
                kind="barh", color="lightgreen", edgecolor="black", ax=axes[1]
            )
        axes[1].set_title(f"{tax_gap_col} by {tax_type_col}")
        axes[1].set_xlabel(f"{tax_gap_col}")
        axes[1].set_ylabel(f"{tax_type_col}")
    else:
        # Plot Audited Declarations (left side) - vertical
        if hue_col:
            tax_type_freq.plot(kind="bar", color=plt.cm.tab20.colors, ax=axes[0])
        else:
            tax_type_freq.plot(
                kind="bar", color="skyblue", edgecolor="black", ax=axes[0]
            )
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel(f"{tax_type_col}")
        axes[0].set_ylabel("Audited Declarations")

        # Plot tax_gap sum (right side) - vertical
        if hue_col:
            tax_gap_sum.plot(kind="bar", color=plt.cm.tab20.colors, ax=axes[1])
        else:
            tax_gap_sum.plot(
                kind="bar", color="lightgreen", edgecolor="black", ax=axes[1]
            )
        axes[1].set_title(f"{tax_gap_col} by {tax_type_col}")
        axes[1].set_xlabel(f"{tax_type_col}")
        axes[1].set_ylabel(f"{tax_gap_col}")

    # Adjust layout for better appearance
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col="tax_type_desc", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col="tax_type_desc", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="tax_type_desc", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_non_audit, tax_type_col="tax_type_desc", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col="tp_size", tax_gap_col="tax_gap")

In [7]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col="tp_size", tax_gap_col="tax_gap")

NameError: name 'visualize_tax_type_and_tax_gap' is not defined

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="tp_size", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_non_audit, tax_type_col="tp_size", tax_gap_col="tax_to_pay_declaration")

The visualizations above show the contribution of different tax types in terms of audit declarations and the tax gap. It is evident that VAT quarterly leads the other tax types in both the number of audit declarations and the amount of the tax gap. Although audit declarations related to CIT are relatively few, the total tax gap graph reveals that CIT ranks second, following VAT quarterly. As for the FLAT regime, the values are quite negligible.

### sectors 

In [ ]:
df['isic_desc'].fillna("Other Service Activities",inplace=True)

In [ ]:
df_non_audit['isic_desc'].fillna("Other Service Activities",inplace=True)

In [8]:
visualize_tax_type_and_tax_gap(df, tax_type_col="isic_desc", tax_gap_col="tax_gap",orientation="horizontal")

NameError: name 'visualize_tax_type_and_tax_gap' is not defined

The graph shows audited declarations and tax gaps by sectors. other service activities has the highest audited declarations at 48.7%, followed by Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles at 27.2%. The largest tax gap is in Other Service Activities (46.9%), followed by Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles (17.7%). Most industries have tax gaps below 5%, with Activities of Extraterritorial Organisations and Bodies at 0% for both metrics.

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="isic_desc", tax_gap_col="tax_to_pay_declaration",orientation="horizontal")

The second graph displays non audited declarations and tax-to-pay declarations by sectors. Other Service Activities lead in declarations at 67.3%, followed by Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles at 12.7%. For tax-to-pay declarations, Other Service Activities again have the highest at 29.1%, with Manufacturing at 13.4%. Most industries have tax-to-pay declarations below 10%, with arts, entertainment and recreation at 0.2%

In [ ]:
def visualize_district_and_tax_gap(
    df, tax_type_col, tax_gap_col, orientation="vertical"
):
    """
    Visualizes the Audited Declarations of tax types and the associated tax gap values side by side.
    You can choose between vertical and horizontal bar charts.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    tax_type_col (str): Column name of the tax type.
    tax_gap_col (str): Column name of the tax gap.
    orientation (str): Orientation of the bar charts ('vertical' or 'horizontal').
    """
    # Calculate the Audited Declarations of each tax_type
    tax_type_freq = df[tax_type_col].value_counts()

    # Calculate the total tax_gap for each tax_type
    tax_gap_sum = df.groupby(tax_type_col)[tax_gap_col].sum()

    # Sort the tax_gap sums in descending order
    tax_gap_sum = tax_gap_sum.sort_values(ascending=False)

    # Create a figure with two subplots (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    if orientation == "horizontal":
        # Plot Audited Declarations (left side) - horizontal
        tax_type_freq.plot(kind="barh", color="skyblue", edgecolor="black", ax=axes[0])
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel("Audited Declarations")
        axes[0].set_ylabel(f"{tax_type_col}")

        # Plot tax_gap sum (right side) - horizontal
        tax_gap_sum.plot(kind="barh", color="lightgreen", edgecolor="black", ax=axes[1])
        axes[1].set_title(f"Total Tax Gap by {tax_type_col}")
        axes[1].set_xlabel("Total Tax Gap")
        axes[1].set_ylabel(f"{tax_type_col}")
    else:
        # Plot Audited Declarations (left side) - vertical
        tax_type_freq.plot(kind="bar", color="skyblue", edgecolor="black", ax=axes[0])
        axes[0].set_title(f"{tax_type_col} Audited Declarations")
        axes[0].set_xlabel(f"{tax_type_col}")
        axes[0].set_ylabel("Audited Declarations")

        # Plot tax_gap sum (right side) - vertical
        tax_gap_sum.plot(kind="bar", color="lightgreen", edgecolor="black", ax=axes[1])
        axes[1].set_title(f"Total Tax Gap by {tax_type_col}")
        axes[1].set_xlabel(f"{tax_type_col}")
        axes[1].set_ylabel("Total Tax Gap")

    # Adjust layout for better appearance
    plt.tight_layout()
    plt.show()

In [ ]:
def district_name(district_no):
    """
    This function maps district number to the district name.

    Parameters
    ----------
    district_no : int
        The district number to get the corresponding district name.

    Returns
    -------
    str or None
        The name of the district if found, otherwise None.
    """
    district_desc = {
        2: "NYARUGENGE",
        6: "KICUKIRO",
        120: "GASABO",
        82: "KAYONZA",
        124: "GATSIBO",
        74: "RWAMAGANA",
        102: "NYAGATARE",
        144: "KIREHE",
        145: "NGOMA",
        146: "BUGESERA",
        148: "KAMONYI",
        121: "NYANZA",
        122: "GISAGARA",
        123: "NYARUGURU",
        129: "NYAMAGABE",
        130: "RUHANGO",
        131: "MUHANGA",
        119: "HUYE",
        125: "MUSANZE",
        139: "RULINDO",
        140: "GAKENKE",
        141: "BURERA",
        142: "GICUMBI",
        127: "KARONGI",
        133: "RUTSIRO",
        134: "RUBAVU",
        135: "NYABIHU",
        136: "NGORORERO",
        137: "RUSIZI",
        138: "NYAMASHEKE",
        1: "OUT OF RWANDA",
    }
    return district_desc.get(district_no, None)

In [ ]:
df["district_name"] = df["district_no"].apply(lambda x: district_name(x))

In [ ]:
df_non_audit["district_name"] = df_non_audit["district_no"].apply(lambda x: district_name(x))

In [ ]:
visualize_district_and_tax_gap(
    df, tax_type_col="district_name", tax_gap_col="tax_gap", orientation="horizontal"
)

In [ ]:
visualize_district_and_tax_gap(
    df_non_audit, tax_type_col="district_name", tax_gap_col="tax_to_pay_declaration", orientation="horizontal"
)

For the district, we observe that Kigali districts are significantly over-contributing compared to the others in terms of audit declarations and the total tax gap.

In [ ]:
# plot_tax_gap_by_audit_type(df, tax_gap_col="tax_gap", audit_type_col="tax_type_desc")

Looking at the graph displaying the distribution of tax gap categories across their corresponding tax types, we can observe that in the "increase" category, VAT outpaces other tax types, indicating that most VAT audited declarations result in a tax gap. On the other hand, in the "no change" category, CIT leads the other tax types.

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col="tp_year", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col="tp_year", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="tp_year", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_non_audit, tax_type_col="tp_year", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(
    df, tax_type_col="audit_type_desc", tax_gap_col="tax_gap"
)

In [9]:
visualize_tax_type_and_tax_gap(
    df_vat_cit_audit, tax_type_col="audit_type_desc", tax_gap_col="tax_gap"
)

NameError: name 'visualize_tax_type_and_tax_gap' is not defined

In [ ]:
visualize_tax_type_and_tax_gap_hue(
    df, tax_type_col="tp_year", tax_gap_col="tax_gap", hue_col="audit_type_desc"
)

In [ ]:
visualize_tax_type_and_tax_gap_hue(
    df_vat_cit_audit, tax_type_col="tp_year", tax_gap_col="tax_gap", hue_col="audit_type_desc"
)

In [ ]:
visualize_tax_type_and_tax_gap_hue(
    df, tax_type_col="tp_size", tax_gap_col="tax_gap", hue_col="audit_type_desc"
)

In [ ]:
df.groupby('audit_type_desc').agg({"hashed_tin":"nunique"}).plot(kind="bar");

## Filling 

In [ ]:
df["submit_date"] = pd.to_datetime(df["submit_date"], errors="coerce")
df["tp_due_date"] = pd.to_datetime(df["tp_due_date"], errors="coerce")
df["late_filling"] = df.apply(
    lambda row: "yes" if row["submit_date"] > row["tp_due_date"] else "no", axis=1
)

In [ ]:
df_vat_cit_audit["submit_date"] = pd.to_datetime(df_vat_cit_audit["submit_date"], errors="coerce")
df_vat_cit_audit["tp_due_date"] = pd.to_datetime(df_vat_cit_audit["tp_due_date"], errors="coerce")
df_vat_cit_audit["late_filling"] = df_vat_cit_audit.apply(
    lambda row: "yes" if row["submit_date"] > row["tp_due_date"] else "no", axis=1
)

In [ ]:
df_non_audit["submit_date"] = pd.to_datetime(df_non_audit["submit_date"], errors="coerce")
df_non_audit["tp_due_date"] = pd.to_datetime(df_non_audit["tp_due_date"], errors="coerce")
df_non_audit["late_filling"] = df_non_audit.apply(
    lambda row: "yes" if row["submit_date"] > row["tp_due_date"] else "no", axis=1
)

In [ ]:
df_vat_cit_non_audit["submit_date"] = pd.to_datetime(df_vat_cit_non_audit["submit_date"], errors="coerce")
df_vat_cit_non_audit["tp_due_date"] = pd.to_datetime(df_vat_cit_non_audit["tp_due_date"], errors="coerce")
df_vat_cit_non_audit["late_filling"] = df_vat_cit_non_audit.apply(
    lambda row: "yes" if row["submit_date"] > row["tp_due_date"] else "no", axis=1
)

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col="late_filling", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col="late_filling", tax_gap_col="tax_gap")

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="late_filling", tax_gap_col="tax_to_pay_declaration")

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_non_audit, tax_type_col="late_filling", tax_gap_col="tax_to_pay_declaration")

The bar charts show that, among all audited declarations, 56% of late filers contribute to 63.2% of the total tax gap.

In [ ]:
plot_tax_gap_by_audit_type(df, tax_gap_col="tax_gap", audit_type_col="late_filling")

In [ ]:
plot_tax_gap_by_audit_type(df_vat_cit_audit, tax_gap_col="tax_gap", audit_type_col="late_filling")

Considering the tax gap categories, we can see that in the "increase" and "no change" categories, late filers are more prevalent. In contrast, the "decrease" category shows a lower proportion of late filers.

In [10]:
df["filling_days"] = (df["submit_date"] - df["tp_due_date"]).dt.days

NameError: name 'df' is not defined

In [ ]:
df_non_audit["filling_days"] = (df_non_audit["submit_date"] - df_non_audit["tp_due_date"]).dt.days

In [ ]:
df_late_filers = df[df["late_filling"] == "yes"]

In [ ]:
df_late_filers_non_audited = df_non_audit[df_non_audit["late_filling"] == "yes"]

In [ ]:
visualize_distribution_and_outliers(df_late_filers, "filling_days")

The graph presents the distribution of the "filling_days (days between submit date and due date") variable for late filers  using a histogram and a box plot. The data appears to be heavily skewed to the right, indicating that a majority of values are concentrated towards the lower end of the range, with a few outliers extending far to the right.

Shape: The histogram shows a clear right-skewed distribution. The majority of values are clustered between 0 and 500 "filling_days", with the frequency decreasing as the number of days increases.

Central Tendency: The peak of the histogram suggests that the most frequent value (mode) lies somewhere between 0 and 100 "filling_days".

Spread: The data is spread out over a wide range, from 0 to approximately 2000 "filling_days".

There are also some outliers 

In [ ]:
visualize_distribution_and_outliers(df_late_filers_non_audited, "filling_days")

### payment 

In [ ]:
df["payment_status"] = df["balance"].apply(lambda x: "yes" if x <= 0 else "no")

In [ ]:
df_vat_cit_audit["payment_status"] = df_vat_cit_audit["balance"].apply(lambda x: "yes" if x <= 0 else "no")

In [ ]:
df_non_audit["payment_status"] = df_non_audit["balance"].apply(lambda x: "yes" if x <= 0 else "no")

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.query('payment_status =="no"')[['tax_gap','balance']]

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col="payment_status", tax_gap_col="tax_gap")

In [11]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col="payment_status", tax_gap_col="tax_gap")

NameError: name 'visualize_tax_type_and_tax_gap' is not defined

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="payment_status", tax_gap_col="tax_to_pay_declaration")

The bar charts show that, among all audited declarations, 60% of non payers contribute to 84.4% of the total tax gap.

In [ ]:
plot_tax_gap_by_audit_type(df, tax_gap_col="tax_gap", audit_type_col="payment_status")

Considering the tax gap categories, we can see that in the "increase" categories non payers are more prevalent. In contrast, the "decrease" and "no change" category shows a lower proportion of non payers.

### claim 

In [ ]:
df["refund_status"] = df["refund_claimed"].apply(
    lambda x: "no" if x == 0 or pd.isna(x) else "yes"
)

In [ ]:
df_vat_cit_audit["refund_status"] = df_vat_cit_audit["refund_claimed"].apply(
    lambda x: "no" if x == 0 or pd.isna(x) else "yes"
)

In [ ]:
df_non_audit["refund_status"] = df_non_audit["refund_claimed"].apply(
    lambda x: "no" if x == 0 or pd.isna(x) else "yes"
)

In [ ]:
visualize_tax_type_and_tax_gap(df, tax_type_col='refund_status', tax_gap_col='tax_gap')

In [ ]:
visualize_tax_type_and_tax_gap(df_vat_cit_audit, tax_type_col='refund_status', tax_gap_col='tax_gap')

In [ ]:
plot_tax_gap_by_audit_type(df, tax_gap_col="tax_gap", audit_type_col="refund_status")

The graphs indicate that the majority of audited declarations, approximately 89%, did not claim a refund. Within the "increase" category, non-refund declarations significantly outnumber refund claim declarations. However, in the other two categories, the trend is reversed, with refund claim declarations being more prevalent.

In [ ]:
visualize_tax_type_and_tax_gap(df_non_audit, tax_type_col="refund_status", tax_gap_col="balance")

### Relationship between some variables 

In [ ]:
df["tax_gap_category"] = df["tax_gap"].apply(
    lambda x: "decrease" if x < 0 else "no change" if x == 0 else "increase"
)

In [ ]:
df_vat_cit_audit["tax_gap_category"] = df_vat_cit_audit["tax_gap"].apply(
    lambda x: "decrease" if x < 0 else "no change" if x == 0 else "increase"
)

##### EBM sales, purchases, imports , exports  vs turnovers 

In [ ]:
df_decrease = df[df["tax_gap_category"] == "decrease"]
df_increase = df[df["tax_gap_category"] == "increase"]
df_no_change = df[df["tax_gap_category"] == "decrease"]

In [ ]:
df_decrease_version = df_vat_cit_audit[df_vat_cit_audit["tax_gap_category"] == "decrease"]
df_increase_version = df_vat_cit_audit[df_vat_cit_audit["tax_gap_category"] == "increase"]
df_no_change_version = df_vat_cit_audit[df_vat_cit_audit["tax_gap_category"] == "decrease"]

In [ ]:
def plot_relationships(df, columns, target_column):
    # Set up the figure with subplots
    num_columns = len(columns)
    fig, axes = plt.subplots(1, num_columns, figsize=(5 * num_columns, 5))

    # Iterate over the columns to create scatter plots
    for i, col in enumerate(columns):
        sns.scatterplot(x=df[col], y=df[target_column], ax=axes[i])
        axes[i].set_xlabel(col)
        axes[i].set_ylabel(target_column)
        axes[i].set_title(f"{col} vs {target_column}")

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_relationships_with_log(df, columns, target_column):
    # Set up the figure with subplots
    num_columns = len(columns)
    fig, axes = plt.subplots(1, num_columns, figsize=(5 * num_columns, 5))

    # Iterate over the columns to create scatter plots
    for i, col in enumerate(columns):
        # Apply logarithmic transformation (log1p handles zeros and negatives)
        log_x = np.log1p(df[col])  # Apply log transformation to the column
        log_y = np.log1p(
            df[target_column]
        )  # Apply log transformation to the target column

        # Plot the scatter plot with log-transformed values
        sns.scatterplot(x=log_x, y=log_y, ax=axes[i])
        axes[i].set_xlabel(f"log({col})")
        axes[i].set_ylabel(f"log({target_column})")
        axes[i].set_title(f"log({col}) vs log({target_column})")

    plt.tight_layout()
    plt.show()

In [ ]:
for i in [df_decrease, df_increase, df_no_change]:
    plot_relationships(
        i, ["ebm_sales", "ebm_purchases", "cif", "fob_amount"], "turnovers"
    )

In [ ]:
for i in [df_decrease, df_increase, df_no_change]:
    plot_relationships_with_log(
        i, ["ebm_sales", "ebm_purchases", "cif", "fob_amount"], "turnovers"
    )

In [ ]:
for i in [df_decrease_version, df_increase_version, df_no_change_version]:
    plot_relationships_with_log(
        i, ["ebm_sales", "ebm_purchases", "cif", "fob_amount"], "turnovers"
    )

In [ ]:
df_non_audit['sales_purchases_balance'] = df_non_audit['ebm_sales'] - df_non_audit['ebm_purchases']

In [12]:
df_non_audit

NameError: name 'df_non_audit' is not defined

In [ ]:
df['sales_purchases_balance'] = df['ebm_sales'] - df['ebm_purchases']

In [ ]:
df["sales_purch_balance_status"] = df["sales_purchases_balance"].apply(lambda x: "purchase_greater" if x <= 0 else "sales_greater")

In [ ]:
df_non_audit["sales_purch_balance_status"] = df_non_audit["sales_purchases_balance"].apply(lambda x: "purchase_greater" if x <= 0 else "sales_greater")

In [ ]:
def plot_bar_with_percentage(df, column):
    """
    Plots a bar chart of value counts for a given column in the DataFrame, 
    with percentages displayed on top of each bar.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column (str): The column name to plot.
    """
    # Compute value counts and percentages
    counts = df[column].value_counts()
    percentages = counts / counts.sum() * 100

    # Plot bar chart
    ax = counts.plot(kind='bar', figsize=(8, 5), color='skyblue')

    # Add percentage labels on bars
    for i, (count, pct) in enumerate(zip(counts, percentages)):
        ax.text(i, count + 0.02 * counts.max(), f'{pct:.1f}%', ha='center', fontsize=12, fontweight='bold')

    # Labels and title
    plt.xlabel('Sales vs Purchases')
    plt.ylabel('declarations')
    plt.title('comparison between sales and purchase for each declaration')
    plt.xticks(rotation=45)
    plt.show()

# Example usage
plot_bar_with_percentage(df_non_audit, 'sales_purch_balance_status')

In [ ]:
plot_bar_with_percentage(df, 'sales_purch_balance_status')

# non exemption sales 

In [ ]:
exempt_non_audit = (df_non_audit.exempted_sales.notna().sum()/ df_non_audit.shape[0]) * 100

In [ ]:
exempt_audit = (df.exempted_sales.notna().sum()/ df.shape[0]) * 100

In [ ]:
exe = [exempt_non_audit,exempt_audit]

In [ ]:
labels = ['non_audited', 'audited']
values = exe

plt.bar(labels, values, color=['blue', 'orange'])

# Remove y-axis
plt.gca().axes.yaxis.set_visible(False)

# Add values above bars
for i, v in enumerate(values):
    plt.text(i, v + 2, f'{v:.2f}', ha='center', fontsize=12, fontweight='bold')

plt.title('Comparison of exemptions sales between audited and non audited declarations');

In [ ]:
df_increase = df[df['tax_gap'] > 0]

In [ ]:
df_decrease = df[df['tax_gap'] < 0]

In [ ]:
df_no_change = df[df['tax_gap']==0]

In [ ]:
increase_ex = (df_increase.query('exempted_sales.notna()').shape[0]/df_increase.shape[0]) * 100

In [ ]:
decrease_ex = (df_decrease.query('exempted_sales.notna()').shape[0]/df_decrease.shape[0]) * 100

In [ ]:
no_ex = (df_no_change.query('exempted_sales.notna()').shape[0]/df_no_change.shape[0]) * 100

In [ ]:
ex_cat = [increase_ex,decrease_ex,no_ex]

In [ ]:
ex_cat

In [ ]:
labels = ['increase', 'decrease','no_change']
values = ex_cat

plt.bar(labels, values, color=['red', 'orange','blue'])

# Remove y-axis
plt.gca().axes.yaxis.set_visible(False)

# Add values above bars
for i, v in enumerate(values):
    plt.text(i, v + 2, f'{v:.2f}', ha='center', fontsize=10, fontweight='bold')

plt.title('Comparison of exemptions sales between audited categories');

### input / output analysis on VAT 

In [ ]:
df_non_audit_vat = df_non_audit.query('tax_type_no in (1,70) & tax_to_pay_declaration > 0')

In [13]:
df_non_audit_vat.head()

NameError: name 'df_non_audit_vat' is not defined

In [ ]:
df_non_audit_vat['ratio'] = df_non_audit_vat['total_vat_paid_on_inputs'] / df_non_audit_vat['vat_on_taxable_sales']

In [ ]:
(df_non_audit_vat.query('ratio > 0.95').shape[0] / df_non_audit_vat.shape[0]) * 100

In [ ]:
df_vat = df.query('tax_type_no in (1,70) & tax_to_pay_declaration > 0')

In [ ]:
df_vat['ratio'] = df_vat['total_vat_paid_on_inputs'] / df_vat['vat_on_taxable_sales']

In [ ]:
(df_vat.query('ratio > 0.95').shape[0] / df_vat.shape[0]) * 100

The image contains four scatter plots that visualize the relationships between different variables and turnovers. Below is the analysis for each plot:

###### ebm_sales vs turnovers:

Observation: The data points are concentrated near the origin (lower values of both ebm_sales and turnovers). A few outliers are visible with very high ebm_sales and turnovers.

Interpretation: For most cases, there seems to be a positive relationship between ebm_sales and turnovers, but the distribution suggests a non-linear pattern with possible noise or outliers.


##### ebm_purchases vs turnovers:

Observation: Similar to the first plot, most data points are concentrated near the origin, with fewer data points showing higher ebm_purchases and turnovers.

Interpretation: A weak positive correlation might exist, but the data's spread indicates variability and potential anomalies.

###### cif vs turnovers:

Observation: Concentration of data points near the origin is prominent, with a few cases of higher cif and turnovers.
Interpretation: The correlation between cif and turnovers appears weak. The sparse distribution of points at higher values might suggest a limited subset of entities with significant values.

##### fob_amount vs turnovers:

Observation: This plot shows the least spread among all four, with most points clustered around very low fob_amount and turnovers values. A few extreme outliers are visible.
Interpretation: The relationship between fob_amount and turnovers seems weaker compared to the other variables, with most data points concentrated at low values.

In [ ]:
for i in [df_decrease, df_increase, df_no_change]:
    plot_relationships(
        df,
        ["ebm_sales", "ebm_purchases", "cif", "fob_amount"],
        "tax_to_pay_declaration",
    )

In [14]:
for i in [df_decrease, df_increase, df_no_change]:
    plot_relationships_with_log(
        df,
        ["ebm_sales", "ebm_purchases", "cif", "fob_amount"],
        "tax_to_pay_declaration",
    )

NameError: name 'df_decrease' is not defined

Visual inspection suggests weak relationships between the variables and tax_to_pay_declaration.


### histogram and boxplots of some numerical columns 

In [ ]:
columns_to_plot = [
    "ebm_sales",
    "ebm_taxable_sales",
    "ebm_exempted_sales",
    "ebm_zero_rated_sales",
    "ebm_vat",
    "ebm_purchases",
    "ebm_taxable_purchases",
    "ebm_exempted_purchases",
    "ebm_zero_rated_purchases",
    "ebm_purchases_vat",
    "turnovers",
    "tax_to_pay"
]

In [ ]:
for i in columns_to_plot:
    visualize_distribution_and_outliers(df, i)

In [ ]:
for i in columns_to_plot:
    visualize_distribution_and_outliers(df_non_audit, i)

In [ ]:
df_columns = df[columns_to_plot]

In [ ]:
df_non_audit_columns = df_non_audit[columns_to_plot]

In [ ]:
# Compute the correlation matrix
corr_matrix = df_columns.corr()

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5);

In [ ]:
# Compute the correlation matrix
corr_matrix_non = df_non_audit_columns.corr()

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix_non, annot=True, cmap='coolwarm', linewidths=0.5);

**These graphs consists of two visualizations representing the distribution of different numerical variables**:

###### Histogram (Left):

The histograms show that the majority of values for those variables are concentrated near zero, with a very high frequency.

There is a significant skew in the data, as most values are small, and very few are large.
This suggests that variables distributions are highly right-skewed, with extreme outliers on the higher end.

###### Box Plot (Right):

The box plot confirms the presence of extreme outliers in the variables, as indicated by the dots far away from the central range of the data.

The interquartile range (IQR), represented by the boxs, are very narrow, showing that most data points lie within a small range near zero.

The red line represents the median, which is also near zero, further indicating the skewness of the data.

**Summary Analysis**:

The variables exhibit a highly skewed distribution with a majority of values near zero.
A small number of extreme outliers exist, representing unusually high amounts in those variables.

Given the disparity between the bulk of the data and the outliers, this variable may require:

**Outlier handling**: Identify and potentially remove or transform extreme values to avoid distortion in further analysis.

##### relationship with target variable 

In [ ]:
def get_relationships(df, target_column):
    # Identify float columns
    float_columns = df.select_dtypes(include=["float64"]).columns.tolist()

    # Create an empty list to store the relationships
    relationships = []

    # Calculate the Pearson correlation between each float column and the target column
    for col in float_columns:
        # Calculate the Pearson correlation between the column and the target column
        correlation = df[col].corr(df[target_column])

        # Store the relationship in the list
        relationship = {"Column": col, "Correlation_with_Target": correlation}

        relationships.append(relationship)

    # Convert the list of relationships into a DataFrame
    relationships_df = pd.DataFrame(relationships)

    return relationships_df

In [ ]:
get_relationships(df, "tax_gap").sort_values(
    by="Correlation_with_Target", ascending=False
).head(10)